In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, models
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16
from tensorflow.keras.optimizers import Adam
from keras_vggface.vggface import VGGFace

In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [3]:
image_path = "./dataset_f+r20/images.npy"
emotion_path = "./dataset_f+r20/emotions_multi.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = layers.Rescaling(1./127.5, offset=-1)(images)
images = tf.image.grayscale_to_rgb(images)

emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

In [4]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [5]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                    loss=loss, 
                    metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [6]:
def create_model(base_model):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

learning_rate = 1e-4
num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()

base_model = VGGFace(model='vgg16', include_top=False, input_shape=(48, 48, 3))
history_save_path = './history/VGGFace.txt'
model = create_model(base_model)
history = train(model, learning_rate, loss, num_epochs, batch_size)
with open(history_save_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

58925056/58909280 [==============================] - 9s 0us/step


C:\Users\Darkl\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(
C:\Users\Darkl\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


Epoch 1/40
Epoch 1/40
3983/3983 [==============================] - 276s 68ms/step - loss: 0.0283 - model_acc: 0.6918 - val_loss: 0.0217 - val_model_acc: 0.7511
Epoch 2/40
3983/3983 [==============================] - 271s 68ms/step - loss: 0.0177 - model_acc: 0.7919 - val_loss: 0.0185 - val_model_acc: 0.7809
Epoch 3/40
3983/3983 [==============================] - 271s 68ms/step - loss: 0.0133 - model_acc: 0.8387 - val_loss: 0.0172 - val_model_acc: 0.8039
Epoch 4/40
3983/3983 [==============================] - 275s 69ms/step - loss: 0.0104 - model_acc: 0.8711 - val_loss: 0.0169 - val_model_acc: 0.8016
Epoch 5/40
3983/3983 [==============================] - 271s 68ms/step - loss: 0.0085 - model_acc: 0.8937 - val_loss: 0.0162 - val_model_acc: 0.8050
Epoch 6/40
3983/3983 [==============================] - 269s 68ms/step - loss: 0.0071 - model_acc: 0.9099 - val_loss: 0.0162 - val_model_acc: 0.8138
Epoch 7/40
3983/3983 [==============================] - 274s 69ms/step - loss: 0.0061 - model_a

3983/3983 [==============================] - 267s 67ms/step - loss: 0.0021 - model_acc: 0.9731 - val_loss: 0.0160 - val_model_acc: 0.8126
Epoch 23/40
3983/3983 [==============================] - 270s 68ms/step - loss: 0.0019 - model_acc: 0.9751 - val_loss: 0.0162 - val_model_acc: 0.8151
Epoch 24/40
3983/3983 [==============================] - 269s 68ms/step - loss: 0.0019 - model_acc: 0.9743 - val_loss: 0.0161 - val_model_acc: 0.8191
Epoch 25/40
3983/3983 [==============================] - 273s 69ms/step - loss: 0.0020 - model_acc: 0.9736 - val_loss: 0.0167 - val_model_acc: 0.8061
Epoch 26/40
3983/3983 [==============================] - 274s 69ms/step - loss: 0.0016 - model_acc: 0.9798 - val_loss: 0.0157 - val_model_acc: 0.8202
Epoch 27/40
3571/3983 [=========================>....] - ETA: 27s - loss: 0.0020 - model_acc: 0.9732